In [1]:
pwd

'/content'

In [2]:
%config IPCompleter.greedy=True

In [ ]:
caminho = 'D:\DeepNLP\base\movie_titles_metadata.txt'

arq = open(r'D:\DeepNLP\base\movie_titles_metadata_new.txt')

In [ ]:
arq.read()

In [ ]:
arq = open()


In [ ]:
arq.seek(0)

In [ ]:
conversas = open(r'D:\DeepNLP\base\movie_conversations.txt', encoding='utf8').read().split('/n')
type(conversas)
conversas

In [ ]:
print(arq.read())

In [9]:
import pandas as pd
import re

In [11]:
#df = pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata.txt',sep=' +++$+++ ')
#df = pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata_new.txt', header=None,  engine= 'python', sep=';')
df = pd.read_csv('https://github.com/lucsantana/movieDlgPT-BR/blob/main/movie_titles_metadata_new.txt'  ,header=None,  sep=';')
# correct Separator should be sep=' +++$+++ ' I've got an error, so I need to clean the results
# I've change the original file to use ";" as Separator

ParserError: ignored

In [ ]:
df.head()


**movie_titles_metadata.txt**
	
    - contains information about each movie title
	- fields: 
		- movieID, 
		- movie title,
		- movie year, 
	   	- IMDB rating,
		- no. IMDB votes,
 		- genres in the format ['genre1','genre2',É,'genreN']

In [ ]:

df.columns = ['movieID','movieTitle','movieYear','IMDBRating','IMDBVotes','genres']
#change  columns name folowing description above

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['subURL']= 'N/A'
#this new column will keep subtitle URL to new corpus.
df['found']= 'N/A'
#this new Column will help to control status


In [ ]:
df.dtypes


**Searching for subtitles**
    I'll get subtitles in OpenSubtitles.org and catalog in pandas dataframe
    

In [10]:
import time
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

ModuleNotFoundError: ignored

In [ ]:
driver = webdriver.Firefox()
driver.get('https://www.opensubtitles.org/')



In [ ]:
pd.set_option('display.max_colwidth',200)
df['subURL']

In [ ]:
# Find subtitles and save them
x = range(len(df))
for i in x :
  
  driver.find_element(By.ID, "search_text").clear()
  driver.implicitly_wait(1)
  driver.find_element(By.ID, "search_text").send_keys(df['movieTitle'][i])
  
  driver.find_element(By.ID, "search_text").send_keys(Keys.ENTER)
  driver.implicitly_wait(6)
  # Check if specific DIV exists to get data instead of list->div class="msg" itemscope="" itemtype="http://schema.org/Movie"
  # Xpath /html/body/div[1]/div[11]
  # CSS Selector = div.msg:nth-child(15)
  element = driver.find_element_by_xpath("/html/body/div[1]/div[11]")
  # Xpath /html/body/div[1]/div[11]
  # CSS Selector = div.msg:nth-child(15) 
  if len(element.text) > 0 : 
     df.loc[i,'found']= 'ok'
  else:
     df.loc[i,'found']= 'nok'
    #not tested from here but can work
     driver.find_element(By.ID, "search_text").clear()
     driver.implicitly_wait(1)
     driver.find_element(By.ID, "search_text").send_keys(df['movieTitle'][i]+' ('+ df['movieYear'][i]+')')
     driver.find_element(By.ID, "search_text").send_keys(Keys.ENTER)
     driver.implicitly_wait(6)
     element = driver.find_element_by_xpath("/html/body/div[1]/div[11]")
     if len(element.text) > 0 : 
       df.loc[i,'found']= 'ok'
     #not tested til here
          
  df.loc[i,'subURL']= driver.current_url
  time.sleep(7)
  print (i) #just to see how many lines was already read
    
driver.quit()

In [ ]:
#save results in CSV file
df.to_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL.csv', encoding='utf-8', sep=';')

**Results and next step**
    #1 - get all records that have in "found" column an "ok" attibute and concatenate at the end of URL "/sort-7/asc-0" to order the results for the most downloaded subtitles then download.
    #2 - get all records that have in "found" column an "nok" attibute and get the correct URLs for the movies, then execute step 1
    #3 - after a visual inspection I've marked some records with "check" atribute, so we need to check manually.

In [ ]:
#I did some changes and a manually inspection on file,and rename it
#everything was marked as "OK" but url had "\sserach\" I change and update manually
#load results to start to work
df_result = pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 1stProcess.csv',  engine= 'python', sep=';')
#df_result = pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 2ndProcess.csv',  engine= 'python', sep=';')


In [ ]:
#some 
df_result.head(10)

In [ ]:
df_result.dtypes

In [ ]:

#create new dataframe to work just with "nok" cases
#found correct URL to download subtitles
df_nok=df_result.query('found =="nok"')


In [ ]:
df_nok.dtypes


In [ ]:
df_nok.reset_index(drop=True, inplace=True)
df_nok.head()

In [ ]:
driver = webdriver.Firefox()

In [ ]:
# find and save corret URL to download page
element = ''
for x in range(len(df_nok)) :
    URL = df_nok['subURL'][x]
    driver.get(URL)
    elementBkp = element
    try :
      element = driver.find_element_by_xpath('//*[@id="search_results"]')
      html_content = element.get_attribute('outerHTML')
      soup = BeautifulSoup(html_content, 'html.parser')
    except:
      element = elementBkp
      #just a Detour

    for i in range(len(soup.find_all('a'))) :        
       if soup.find_all('a')[i].get_text().replace('\n',' ').lower().find((df_nok['movieTitle'][x]+' ('+df_nok['movieYear'][x]+')').lower())>= 0 : 
          print(soup.find_all('a')[i],'\n') 
          #print('https://www.opensubtitles.org'+soup.find_all('a')[i].get('href'))
          df_result.loc[df_nok['Unnamed: 0'][x],'found']= 'upd'
          df_result.loc[df_nok['Unnamed: 0'][x],'subURL']= 'https://www.opensubtitles.org'+soup.find_all('a')[i].get('href')  
          break 
            

In [ ]:
df_result.to_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 2ndProcess.csv', encoding='utf-8', sep=';')

**Load results**

I did some changes mannualy just to check if everything is OK.

I'm going to download those subtitles, that I already have URLs, save and clean those files.

In [ ]:
df_result = pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 2ndProcess.csv',  engine= 'python', sep=';')

In [ ]:
df_ok=df_result.query('found !="nok"')

In [ ]:
df_ok.reset_index(drop=True, inplace=True)
df_ok.tail()

,Unnamed: 0,movieID,movieTitle,movieYear,IMDBRating,IMDBVotes,genres,subURL,found
574,612,m612,watchmen,2009,7.8,135229,"['action', 'crime', 'fantasy', 'mystery', 'sci...",https://www.opensubtitles.org/pt/search/sublan...,ok
575,613,m613,xxx,2002,5.6,53505,"['action', 'adventure', 'crime']",https://www.opensubtitles.org/pt/search/sublan...,ok
576,614,m614,x-men,2000,7.4,122149,"['action', 'sci-fi']",https://www.opensubtitles.org/pt/search/sublan...,ok
577,615,m615,young frankenstein,1974,8.0,57618,"['comedy', 'sci-fi']",https://www.opensubtitles.org/pt/search/sublan...,ok
578,616,m616,zulu dawn,1979,6.4,1911,"['action', 'adventure', 'drama', 'history', 'w...",https://www.opensubtitles.org/pt/search/imdbid...,ok


In [ ]:

#create a new column to keep file URL.
df_ok['subDownUrl']= 'N/A'


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_ok.dtypes

Unnamed: 0      int64
movieID        object
movieTitle     object
movieYear       int64
IMDBRating    float64
IMDBVotes       int64
genres         object
subURL         object
found          object
subDownUrl     object
dtype: object

In [8]:
import time
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC


ModuleNotFoundError: ignored

In [ ]:
#1 concatenar "/sort-7/asc-0" no final da URL
element = ''

c = DesiredCapabilities.FIREFOX
c["pageLoadStrategy"] = "none"

driver = webdriver.Firefox(desired_capabilities=c)
#driver = webdriver.Firefox()
for x in range(len(df_ok)) :
    w = WebDriverWait(driver, 15)
    URL = df_ok['subURL'][x]+'/sort-7/asc-0' # get the most downloaded subtitle.
    print(str(x)+URL )
    driver.get(URL)
    elementBkp = element
    try :
    
     #release the process as soon as locate the element we are looking for.
      w.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/form/table/tbody/tr[2]/td[1]/strong/a')))
      element = driver.find_element_by_xpath('/html/body/div[1]/form/table/tbody/tr[2]/td[1]/strong/a') 
      #get the exactly right Xpath to link.
      html_content = element.get_attribute('outerHTML')
      soup = BeautifulSoup(html_content, 'html.parser')
      print('passou aqui')
    except:
      element = elementBkp
      #just a Detour

    #passo 2 Achar as URLs corretas para trabalhar 

    try:
      df_ok.loc[x,'subDownUrl']= 'https://www.opensubtitles.org'+soup.find('a').get('href')
      df_ok.loc[x,'found']= 'dwl'
      print(soup.find('a').get('href'))
      print('indice - '+ str(x))
    except:
      df_ok.loc[x,'found']= 'err'
      print('Erro Linha'+ str(x))
    
    #2 criar processo de download e gravação dos arquivo
    time.sleep(7)  
driver.close()
df_ok.to_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 3rdProcess.csv', encoding='utf-8', sep=';')

## One last thing
the steps above does not bring correct URL, so we have to discovery correct link


In [ ]:
import time
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
#pegar a verdadeira url de download
element = ''
c = DesiredCapabilities.FIREFOX

c["pageLoadStrategy"] = "none"

driver = webdriver.Firefox(desired_capabilities=c)

for x in range(len(df_ok)) :
    w = WebDriverWait(driver, 15)
    URL = df_ok['subDownUrl'][x] # get the real subtitle URL.
    print(str(x)+URL )
    driver.get(URL)
    elementBkp = element
    try :    
     #release the process as soon as locate the element we are looking for.
      w.until(EC.presence_of_element_located((By.XPATH, '//*[@id="bt-dwl-bt"]')))
      element = driver.find_element_by_xpath('//*[@id="bt-dwl-bt"]') 
      #get the exactly right Xpath to link.
      html_content = element.get_attribute('outerHTML')
      soup = BeautifulSoup(html_content, 'html.parser')
      print('passou aqui')
    except:
      element = elementBkp

    try:
      df_ok.loc[x,'subDownUrl']= 'https://www.opensubtitles.org'+ soup.find(id="bt-dwl-bt").get('href')
      df_ok.loc[x,'found']= 'file'
      print(soup.find('a').get('href'))
      print('indice - '+ str(x))
    except:
      df_ok.loc[x,'found']= 'err'
      print('Erro Linha'+ str(x))
    
    #2 criar processo de download e gravação dos arquivo
    time.sleep(7)
driver.close()
df_ok.to_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 4thProcess.csv', encoding='utf-8', sep=';')

**Post Processing**
After process and finally get all subtile URLs, I check and change some lines manually, now I'll join them

In [ ]:

df_ok=pd.read_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - 4thProcess.csv',  engine= 'python', sep=';')

In [ ]:
df_ok.reset_index(drop=True, inplace=True)


In [ ]:
df_full = df_result.merge(df_ok[['movieID','found','subDownUrl']], left_on='movieID', right_on='movieID' , how='left')

In [ ]:
#df_result.drop(labels='subDownUrl', axis=1,inplace=True)
#df_full.drop(labels='found_y', axis=1,inplace=True)
df_full.dtypes

Unnamed: 0      int64
movieID        object
movieTitle     object
movieYear       int64
IMDBRating    float64
IMDBVotes       int64
genres         object
subURL         object
found_x        object
subDownUrl     object
dtype: object

In [ ]:
df_full.columns = ['index','movieID','movieTitle','movieYear','IMDBRating','IMDBVotes','genres','subURL','found','subDownUrl']

In [ ]:
df_full.to_csv(r'D:\DeepNLP\base\movie_titles_metadata_URL - Final.csv', encoding='utf-8', sep=';')

In [10]:
df_full = pd.read_csv('https://raw.githubusercontent.com/lucsantana/movieDlgPT-BR/main/movie_titles_metadata_URL%20-%20Final.csv', sep=';')

In [ ]:
df_full.query('found=="nok"')

In [12]:
 pd.set_option("display.max_colwidth", 300)

# Download and catalog files



Step By Step


1.   Include a new Field in Dataframe in order to save subtitle file name. - OK
2.   Download subtitles Files - OK
3.   Check file extention
4.   If a Zipped file extract the file and catalog it to further manipulation.



In [50]:
df_full['fileName']= 'N/A'
df_full['subFileName']='N/A'

In [51]:
df_full.dtypes

index            int64
movieID         object
movieTitle      object
movieYear        int64
IMDBRating     float64
IMDBVotes        int64
genres          object
subURL          object
found           object
subDownUrl      object
fileName        object
subFileName     object
dtype: object

In [16]:
df_full['subDownUrl'].head()



0    https://www.opensubtitles.org/pt/subtitleserve/sub/4253498
1    https://www.opensubtitles.org/pt/subtitleserve/sub/3141352
2     https://www.opensubtitles.org/pt/subtitleserve/sub/138875
3    https://www.opensubtitles.org/pt/subtitleserve/sub/3173863
4      https://www.opensubtitles.org/pt/subtitleserve/sub/84899
Name: subDownUrl, dtype: object

In [19]:
#!pip install wget

In [20]:
import wget
from zipfile import ZipFile

In [55]:
# Download zip files and extract subtitles from it.
# keep track in two new fields "fileName", "subFileName"

print('Beginning file download with wget module')
for x in range(len(df_full.head(10))) : #getting first 10 entries to test the process.
    url = df_full.loc[x,'subDownUrl']
    print(url+' - '+df_full.loc[x,'movieTitle'])
    file = wget.download(url, out='/content/sub')  
    zip = ZipFile(file, 'r')
    for i in range(len(zip.namelist())) :
        if re.search('srt$|sub$',str(zip.namelist()[i]).lower()) : # find the exact file in zip file based on file extension
           print(zip.namelist()[i])
           zip.extract(path='/content/sub' ,member=zip.namelist()[i], pwd=None)
           df_full.loc[x,'subFileName'] ='/content/sub/'+str(zip.namelist()[i].lower())     
    df_full.loc[x,'fileName'] = file
print('Download is finished')

Beginning file download with wget module
https://www.opensubtitles.org/pt/subtitleserve/sub/4253498 - 10 things i hate about you
10.Things.I.Hate.About.You.1999.720p.BluRay.x264-SiNNERS.srt
https://www.opensubtitles.org/pt/subtitleserve/sub/3141352 - 1492: conquest of paradise
1492 - The Conquest Of Paradise.srt
https://www.opensubtitles.org/pt/subtitleserve/sub/138875 - 15 minutes
15 Minutes - by Pentha.srt
https://www.opensubtitles.org/pt/subtitleserve/sub/3173863 - 2001: a space odyssey
2001.A.Space.Odyssey.1968.720p.HDDVD.x264-hV.srt
https://www.opensubtitles.org/pt/subtitleserve/sub/84899 - 48 hrs.
48 Hrs.ubapraia74.sub
https://www.opensubtitles.org/pt/subtitleserve/sub/4958459 - the fifth element
The.Fifth.Element.Remastered.1997.1080p.BrRip.x264.YIFY.srt
https://www.opensubtitles.org/pt/subtitleserve/sub/3659627 - 8mm
8MM 1999 XviD ENG.srt
https://www.opensubtitles.org/pt/subtitleserve/sub/44143 - a nightmare on elm street 4: the dream master
Hora do Pesadelo 4, A.sub
https://ww

In [56]:
df_full[['fileName', 'subFileName']].head()

,fileName,subFileName
0,/content/sub/10.things.i.hate.about.you.(1999).pob.1cd.(4253498).zip,/content/sub/10.things.i.hate.about.you.1999.720p.bluray.x264-sinners.srt
1,/content/sub/1492.conquest.of.paradise.(1992).pob.1cd.(3141352).zip,/content/sub/1492 - the conquest of paradise.srt
2,/content/sub/15.minutes.(2001).pob.1cd.(138875).zip,/content/sub/15 minutes - by pentha.srt
3,/content/sub/2001.a.space.odyssey.(1968).pob.1cd.(3173863).zip,/content/sub/2001.a.space.odyssey.1968.720p.hddvd.x264-hv.srt
4,/content/sub/48.hrs.(1982).pob.1cd.(84899).zip,/content/sub/48 hrs.ubapraia74.sub


# Cleaning Subtitles files

In [58]:
#zip.filelist[0]
#len(zip.namelist())-1
#checar as extençoes das legendas e extrair do zip
#ZipFile.extractall(path=None, members=nome doarquivo especifico, pwd=None)
#zip.namelist()[0]
#type(zip.namelist())




**Regex** 
  
Using regex to clean subtitles files.

In [64]:
textSub="asdfas"# load SUB file
textSrt= " asdfasdf" # load SRT File

In [65]:
re.search('(..):(..):(..),(...) --> (..):(..):(..),(...)', textSrt) #if found anything, delete this line! SRT.
re.search('^(\d[0-9]*)$', textSrt) #if found anything, delete this line! SRT.

re.search('^{(\d{4,7})}{(\d{4,7})}', textSub) #if found anything, delete  this part! SUB.

re.search('^\s*$',textSub) #identifica as linhas em branco. Devemos apagar todas.

#tentar este codigo para deletar linhas em branco
if re.match(r'^\s*$', line):
    # line is empty (has only the following: \t\n\r and whitespace)
filtered = filter(lambda x: not re.match(r'^\s*$', x), original)

